In [2]:
from flask import render_template, request
from app import app
from models import Historical, Forecast, Station
import datetime

import plotly
import plotly.graph_objs as go 

import pandas as pd
import numpy as np 
import json

In [12]:

def index():   

    line = create_plot()
    return render_template("index.html", plot=line)    
    
def create_plot():
 	N = 40
 	today = datetime.date.today()
 	x1 = [x.id for x in Historical.query.filter(Historical.id >= today - datetime.timedelta(days=7)).order_by(Historical.id)]
 	y1 = [x.drybulb for x in Historical.query.filter(Historical.id >= today - datetime.timedelta(days=7)).order_by(Historical.id)]

 	x2 = [x.id for x in Forecast.query.all()]
 	y2 = [x.drybulb for x in Forecast.query.all()]

 	historicals = go.Line(
 			x=x1,
 			y=y1,
 			name='historicals'
 			)

 	forecasts = go.Line(
 			x=x2,
 			y=y2,
 			name='forecasts'
 			)

 	data = [historicals, forecasts]
 	graphJSON = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)

 	return graphJSON

In [13]:
create_plot()

'[{"x": ["2019-06-12", "2019-06-12 01:00:00", "2019-06-12 02:00:00", "2019-06-12 03:00:00", "2019-06-12 04:00:00", "2019-06-12 05:00:00", "2019-06-12 06:00:00", "2019-06-12 07:00:00", "2019-06-12 08:00:00", "2019-06-12 09:00:00", "2019-06-12 10:00:00", "2019-06-12 11:00:00", "2019-06-12 12:00:00", "2019-06-12 13:00:00", "2019-06-12 14:00:00", "2019-06-12 15:00:00", "2019-06-12 16:00:00", "2019-06-12 17:00:00", "2019-06-12 18:00:00", "2019-06-12 19:00:00", "2019-06-12 20:00:00", "2019-06-12 21:00:00", "2019-06-12 22:00:00", "2019-06-12 23:00:00", "2019-06-13", "2019-06-13 01:00:00", "2019-06-13 02:00:00", "2019-06-13 03:00:00", "2019-06-13 04:00:00", "2019-06-13 05:00:00", "2019-06-13 06:00:00", "2019-06-13 07:00:00", "2019-06-13 08:00:00", "2019-06-13 09:00:00", "2019-06-13 10:00:00", "2019-06-13 11:00:00", "2019-06-13 12:00:00", "2019-06-13 13:00:00", "2019-06-13 14:00:00", "2019-06-13 15:00:00", "2019-06-13 16:00:00", "2019-06-13 17:00:00", "2019-06-13 18:00:00", "2019-06-13 19:00:00